# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Importing Dataset

In [2]:
df= pd.read_csv("hw1_q2_dataset.csv",header = None)
X = df.drop([4],axis=1)
X = np.array(X)
Y_train = df[4]
Y_train = np.array(Y_train)


# Giving Integers To Labels

In [3]:
def trltn(label):
    cols = []
    for i in range(0,len(label)):
        if label[i] == ' Metal':
                cols.append(0)
        elif label[i] == ' Ceramic':
                cols.append(1)
        elif label[i] == ' Plastic':
                cols.append(2)
    return cols

In [4]:
Y_train = df[4]
df = df.drop([4],axis=1)
e = df
e = np.array(e)
Y = trltn(Y_train)
df['labels'] = Y

In [5]:
X = df
X = np.array(X)

# Splitting the Dataset in Ds and Du

In [229]:
Ds = []
l = []
c0=0
c1=0
c2=0
for i in range(0,len(X)):
    if Y[i]==0 and c0<=9:
        c0=c0+1
        Ds.append(X[i])
        l.append(i)
    if Y[i]==1 and c1<=9:
        c1 = c1+1
        Ds.append(X[i])
        l.append(i)
    if Y[i]==2 and c2<=9:
        c2 = c2+1
        Ds.append(X[i])
        l.append(i)
Ds = np.array(Ds)
Y_ds = Ds[:,-1]
Du = []
for i in range(0,len(X)):
    if i not in l:
        Du.append(X[i])
        
Du = np.array(Du)
Y_du = Du[:,-1]
Du = np.delete(Du,len(Du[0])-1,axis=1)
Y_test = np.hstack((Y_ds,Y_du))

# Implementing KNN For Self Training

In [230]:
def predict_knn(Ds,Du,k):
    x_train = Ds
    y_train = x_train[:,-1]
    test_i = Du
    train  = []
    for i in range(0,len(x_train)):
        cols = []
        for j in range(0,len(x_train[i])-1):
            cols.append(x_train[i][j])
        train.append(cols)
    train = np.array(train)
    dist_index = []
    for i in range(0,len(test_i)):
        diff = train-test_i[i]
        diff = np.square(diff)
        diff = np.sum(diff,axis=1)
        diff = np.sqrt(diff)
        index = np.argsort(diff)
        dist_index.append(index)
    labels_dist = []
    for i in range(0,len(dist_index)):
        cols = []
        for j in range(0,k):
            cols.append(y_train[dist_index[i][j]])
        labels_dist.append(cols)
    predict = []
    for i in range(0,len(labels_dist)):
        c1=0
        c2=0
        c0=0
        cols = []
        for j in range(0,len(labels_dist[i])):
            if labels_dist[i][j] == 0:
                c0 = c0+1
            elif labels_dist[i][j] == 1:
                c1 = c1+1
            elif labels_dist[i][j] == 2:
                c2 = c2+2
        if c1==c2 and c2==c0:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==1 or labels_dist[i][j]==2 or labels_dist[i][j]==0:
                    k = labels_dist[i][j]
                    break
        elif c1>c2 and c1>c0:
            k = 1
        elif c2>c1 and c2>c0:
            k=2
        elif c0>c1 and c0>c2:
            k = 0
        elif c0==c1 and c1>c2:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==1 or labels_dist[i][j]==0:
                    k = labels_dist[i][j]
                    break
        elif c1==c2 and c1>c0:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==1 or labels_dist[i][j]==2:
                    k = labels_dist[i][j]
                    break
        elif c0==c2 and c0>c2:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==2 or labels_dist[i][j]==0:
                    k = labels_dist[i][j]
                    break
        cols.append(k)
        predict.append(cols)
    predict = np.array(predict)
    tr = test_i
    tr = np.hstack((tr, predict))
    x_train = np.vstack((x_train,tr))
    return x_train

def accuracy(y,y_t):
    c=0
    for i in range(0,len(y)):
        if y[i]==y_t[i]:
            c=c+1
    c = c/len(y)
    c = c*100
    return c

# Implementing Self-Learning 

In [231]:
def ssl(x_t,x_te,k,n):
    c = len(x_te)
    while(c!=0):
        if k>len(x_te):
            x_t = predict_knn(x_t,x_te,n)
            x_te = []
            c=len(x_te)
        else :
            d = []
            for i in range(0,k):
                d.append(x_te[i])
            h= np.array(d)
            x_t = predict_knn(x_t,d,n)
            d = []
            for i in range(0,len(x_te)):
                if k-1<i:
                    d.append(x_te[i])
            h = np.array(d)
            x_te = h
            c = len(x_te)
    return x_t


# Implementing the Self Learning For Different Neighbors And Number Of Data Points

In [232]:
k = [3,5]
dp = [5,15,20,25]
for i in range(0,len(k)):
    for j in range(0,len(dp)):
        kn = k[i]
        p = dp[j]
        predict = ssl(Ds,Du,p,kn)
        pre = predict[:,-1]
        pre = pre[30:len(pre)]
        y = Y_test[30:len(Y_test)]
        acc = accuracy(pre,y)
        print("The accuracy for number of data points added in each iteration "+str(p)+" and nearest neighbors "+ str(kn)+" is "+str(acc))

        

The accuracy for number of data points added in each iteration 5 and nearest neighbors 3 is 52.22222222222223
The accuracy for number of data points added in each iteration 15 and nearest neighbors 3 is 58.88888888888889
The accuracy for number of data points added in each iteration 20 and nearest neighbors 3 is 56.666666666666664
The accuracy for number of data points added in each iteration 25 and nearest neighbors 3 is 58.88888888888889
The accuracy for number of data points added in each iteration 5 and nearest neighbors 5 is 61.111111111111114
The accuracy for number of data points added in each iteration 15 and nearest neighbors 5 is 58.88888888888889
The accuracy for number of data points added in each iteration 20 and nearest neighbors 5 is 52.22222222222223
The accuracy for number of data points added in each iteration 25 and nearest neighbors 5 is 50.0


# Observation Q2a)

As we can see from the above result that we using self training on KNN Model we get highest accuracy on adding 5 data points that is 61.14% for 5 nearest neigbors , where accuracy is checked on the unlabelled data

# Q2b)

# Implementing A Normal KNN 

In [227]:
def predict_nknn(X_train_df,X_test_df,Y_train,k):
    dist_index = []
    for i in range(0,len(X_test_df)):
        diff = X_train_df - X_test_df[i]
        diff = np.square(diff)
        diff = np.sum(diff,axis=1)
        diff = np.sqrt(diff)
        index = np.argsort(diff)
        dist_index.append(index)
    labels_dist = []
    for i in range(0,len(dist_index)):
        cols = []
        for j in range(0,k):
            cols.append(Y_train[dist_index[i][j]])
        labels_dist.append(cols)

    predict = []
    for i in range(0,len(labels_dist)):
        c1=0
        c2=0
        c0=0
        cols = []
        for j in range(0,len(labels_dist[i])):
            if labels_dist[i][j] == 0:
                c0 = c0+1
            elif labels_dist[i][j] == 1:
                c1 = c1+1
            elif labels_dist[i][j] == 2:
                c2 = c2+2
        if c1==c2 and c2==c0:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==1 or labels_dist[i][j]==2 or labels_dist[i][j]==0:
                    k = labels_dist[i][j]
                    break
        elif c1>c2 and c1>c0:
            k = 1
        elif c2>c1 and c2>c0:
            k=2
        elif c0>c1 and c0>c2:
            k = 0
        elif c0==c1 and c1>c2:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==1 or labels_dist[i][j]==0:
                    k = labels_dist[i][j]
                    break
        elif c1==c2 and c1>c0:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==1 or labels_dist[i][j]==2:
                    k = labels_dist[i][j]
                    break
        elif c0==c2 and c0>c2:
            for j in range(0,len(labels_dist[i])):
                if labels_dist[i][j]==2 or labels_dist[i][j]==0:
                    k = labels_dist[i][j]
                    break
        cols.append(k)
        predict.append(cols)
    return predict


# Implementing The Normal KNN And Finding Accuracy On Different Neighbors

In [240]:
ff = pd.DataFrame(data=Ds)
ff =ff.drop([4],axis=1)
X_N_train = np.array(ff)
predict = predict_nknn(X_N_train,Du,Y_ds,3)
acc = accuracy(predict,Y_test[30:len(Y_test)])
print("Accuracy for Normal KNN we get accuracy for 3 neighbors " +str(acc))

Accuracy for Normal KNN we get accuracy for 3 neighbors 53.333333333333336


# Implementing The Self Training Model In Order To Compare With Normal KNN

## For Four Data Points Added

In [250]:
predict = ssl(Ds,Du,4,3)
pre = predict[:,-1]
pre = pre[30:len(pre)]
y = Y_test[30:len(Y_test)]
acc = accuracy(pre,y)
print("Accuracy for self Training KNN model where data points added after every iteration are 4 have "+str(acc))

Accuracy for self Training KNN model where data points added after every iteration are 4 have 55.55555555555556


## For All Data Points Added

In [249]:
predict = ssl(Ds,Du,len(Du),3)
pre = predict[:,-1]
pre = pre[30:len(pre)]
y = Y_test[30:len(Y_test)]
acc = accuracy(pre,y)
print("Accuracy for self Training KNN model where all data points are added in single iteration have "+str(acc))

Accuracy for self Training KNN model where all data points are added in single iteration have 53.333333333333336


## For Adding Five Points 

In [248]:
predict = ssl(Ds,Du,5,3)
pre = predict[:,-1]
pre = pre[30:len(pre)]
y = Y_test[30:len(Y_test)]
acc = accuracy(pre,y)
print("Accuracy for self Training KNN model where data points added after every iteration are 5 have "+str(acc))

Accuracy for self Training KNN model where data points added after every iteration are 5 have 52.22222222222223


## For Adding Three points (Personal Choice)

In [247]:
predict = ssl(Ds,Du,3,3)
pre = predict[:,-1]
pre = pre[30:len(pre)]
y = Y_test[30:len(Y_test)]
acc = accuracy(pre,y)
print("Accuracy for self Training KNN model where data points added after every iteration are 3 have "+str(acc))

Accuracy for self Training KNN model where data points added after every iteration are 3 have 56.666666666666664


# Outcome Q2b)

From above we can see that accuracy of normal KNN and the accuracy when we added all datapoints in a single iteration is same as both will follow the same procedure only.<br>
From above experiment we can say that when we add 3 and 4 data points the accuracy slightly edges to Normal KNN as both have accuracies of 55.55 and 56.66 respectievely compared to Normal KNN accuracy of 53.33
From above experiment when we add 5 points in every iteration we get less accuracy than the accuracy of Normal KNN

# Conclusion

Thus we can say that whenever we add less number of points we get more accuracy then normal KNN as the model is trained by itself and is gaining knowledge in every iteration thats why the lesser the data point in inserted in every iteration the more will be the accuracy